In [ ]:
import warnings
warnings.filterwarnings('ignore')

import torch
import random
import numpy as np
import torch.backends.cudnn as cudnn

def seed_everything(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    cudnn.benchmark = False
    cudnn.deterministic = True
    random.seed(seed)
    
SEED = 333
seed_everything(SEED)

In [ ]:
from src.dataload import get_inshop_dataset
from sklearn.model_selection import train_test_split

DATASET_DIR = '../../../dataset/InShop/'
all_data, label_enc, cat_enc, subcat_enc, vari_enc = get_inshop_dataset(root_path=DATASET_DIR)

X_train, X_valid, y_train, y_valid, cat_tr, cat_val = train_test_split(all_data, 
                                                            label_enc, 
                                                            cat_enc,
                                                            test_size=0.15, 
                                                            random_state=SEED, 
                                                            stratify=cat_enc)

X_valid, X_test, y_valid, y_test, _, _ = train_test_split(X_valid, 
                                                            y_valid, 
                                                            cat_val,
                                                            test_size=0.5, 
                                                            random_state=SEED, 
                                                            stratify=cat_val)

In [ ]:
from src.utils import MyDataset, TransformsSimCLR

size = 224
batch_size = 32

train_dataset = MyDataset(img_list=X_train, 
                          label_list=y_train, 
                          phase='train', 
                          transform=TransformsCE(size))

val_dataset = MyDataset(img_list=X_valid, 
                          label_list=y_valid, 
                          phase='val', 
                          transform=TransformsCE(size))

test_dataset = MyDataset(img_list=X_test, 
                          label_list=y_test, 
                          phase='test', 
                          transform=TransformsCE(size))

image_datasets = {'train' : train_dataset, 'val' : val_dataset, 'test': test_dataset}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val', 'test']}


train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
                                                                                                                                             
val_dataloader = torch.utils.data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False, drop_last=True)

test_dataloader = torch.utils.data.DataLoader(
    test_dataset, batch_size=batch_size, shuffle=False, drop_last=True)

dataloaders = {'train': train_dataloader, 'val': val_dataloader, 'test': test_dataloader}

-----
### HIE

In [ ]:
SEED = 333
seed_everything(SEED)

torch.cuda.empty_cache()
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

SAVE_PATH = '../../../weights'

RUN_NAME = 'ResNet50-HIE-333'
WANDB_PRJ = 'DeepFashion-pre'

CONFIG = {
    "dataset": "DeepFashion",
    "machine": "offline cluster",
    "model": "ResNet50",
    "learning_rate": 0.2,
    "batch_size": batch_size*2
}

print(f'Training on {device} - {WANDB_PRJ} / {RUN_NAME}')

In [ ]:
import timm

import torch.nn as nn

from src.models import MLP_z
from src.optim_lars import LARS
from src.loss_func import CustomLoss

num_classes = 0
model = timm.create_model('resnet50', pretrained=True, num_classes=num_classes)

mlp = MLP_z(num_classes=17) 

criterion = CustomLoss(batch_size=batch_size, temperature=0.1)

optimizer = LARS(
    list(model.parameters()+list(mlp.parameters()),
    lr=0.2,
    weight_decay=1e-6,
    exclude_from_weight_decay=['batch_normalization', 'bias'],
)

warmupscheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lambda epoch : (epoch+1)/10.0, verbose = True)
mainscheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, 500, eta_min=0.05, last_epoch=-1, verbose = True)

In [ ]:
import wandb

from src.train import run_training


wandb.init(name=RUN_NAME, project=WANDB_PRJ, config=CONFIG, reinit=True)

run_training(model, mlp, dataloaders, dataset_sizes,
             optimizer, criterion, warmupscheduler, mainscheduler, device, RUN_NAME, SAVE_PATH, START=0, END=501)

wandb.finish()

-----
### NTXent

In [ ]:
SEED = 333
seed_everything(SEED)

torch.cuda.empty_cache()
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

SAVE_PATH = '../../../weights'

RUN_NAME = 'ResNet50-SupCL-333'
WANDB_PRJ = 'DeepFashion-pre'

CONFIG = {
    "dataset": "DeepFashion",
    "machine": "offline cluster",
    "model": "ResNet50",
    "learning_rate": 0.2,
    "batch_size": batch_size*2
}

print(f'Training on {device} - {WANDB_PRJ} / {RUN_NAME}')

In [ ]:
import timm

import torch.nn as nn

from src.models import MLP_z
from src.optim_lars import LARS
from src.loss_func import NT_Xent

num_classes = 0
model = timm.create_model('resnet50', pretrained=True, num_classes=num_classes)

mlp = MLP_z(num_classes=17) 

criterion = NT_Xent(batch_size=batch_size, temperature=0.1)

optimizer = LARS(
    list(model.parameters()+list(mlp.parameters()),
    lr=0.2,
    weight_decay=1e-6,
    exclude_from_weight_decay=['batch_normalization', 'bias'],
)

warmupscheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lambda epoch : (epoch+1)/10.0, verbose = True)
mainscheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, 500, eta_min=0.05, last_epoch=-1, verbose = True)

In [ ]:
import wandb

from src.train import run_training


wandb.init(name=RUN_NAME, project=WANDB_PRJ, config=CONFIG, reinit=True)

run_training(model, mlp, dataloaders, dataset_sizes,
             optimizer, criterion, warmupscheduler, mainscheduler, device, RUN_NAME, SAVE_PATH, START=0, END=501)

wandb.finish()

-------
### SupCL

In [ ]:
SEED = 333
seed_everything(SEED)

torch.cuda.empty_cache()
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

SAVE_PATH = '../../weights'

RUN_NAME = 'ResNet50-SupCL-333'
WANDB_PRJ = 'DeepFashion-pre'

CONFIG = {
    "dataset": "DeepFashion",
    "machine": "offline cluster",
    "model": "ResNet50",
    "learning_rate": 0.2,
    "batch_size": batch_size*2
}

print(f'Training on {device} - {WANDB_PRJ} / {RUN_NAME}')

In [ ]:
import timm

import torch.nn as nn

from src.models import MLP_z
from src.optim_lars import LARS
from src.loss_func import SupConLoss

num_classes = 0
model = timm.create_model('resnet50', pretrained=True, num_classes=num_classes)

mlp = MLP_z(num_classes=17) 

criterion = SupConLoss(batch_size=batch_size, temperature=0.1)

optimizer = LARS(
    list(model.parameters()+list(mlp.parameters()),
    lr=0.2,
    weight_decay=1e-6,
    exclude_from_weight_decay=['batch_normalization', 'bias'],
)

warmupscheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lambda epoch : (epoch+1)/10.0, verbose = True)
mainscheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, 500, eta_min=0.05, last_epoch=-1, verbose = True)

In [ ]:
import wandb

from src.train import run_training


wandb.init(name=RUN_NAME, project=WANDB_PRJ, config=CONFIG, reinit=True)

run_training(model, mlp, dataloaders, dataset_sizes,
             optimizer, criterion, warmupscheduler, mainscheduler, device, RUN_NAME, SAVE_PATH, START=0, END=501)

wandb.finish()